In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load and prepare data (just ignore)

Load the data from up-to-date online sources, combine them into a single dataframe

In [6]:
df=pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')

df=df.iloc[::-1]
df['Date']=df['date'].astype(str).str[5:]
df['DateTime']=pd.to_datetime(df['date'], format='%Y%m%d')
df=df.fillna(0)

tests_agged=df.groupby(['state']).agg({'totalTestResultsIncrease':np.sum}).reset_index()

abb=pd.read_csv('http://www.fonz.net/blog/wp-content/uploads/2008/04/states.csv')
abb_to_full_mapper={a:b for a,b in zip(abb['Abbreviation'], abb['State'])}

pop=pd.read_csv('http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv', usecols=['NAME', 'CENSUS2010POP'])
full_to_pop_mapper={a:b for a,b in zip(pop['NAME'], pop['CENSUS2010POP'])}

df['Population']=df['state'].map(abb_to_full_mapper).map(full_to_pop_mapper)
df['Per100kDeath']=df['deathIncrease']/df['Population']*100000
df['Per100kTests']=df['totalTestResultsIncrease']/df['Population']*100000
df['Per100kDeathCumulative']=df['death']/df['Population']*100000
df['Per100kTestsCumulative']=df['totalTestResults']/df['Population']*100000

df=df.dropna()
df=df[['date', 'state', 'Population', 'Per100kDeath', 'Per100kTests', 'Per100kDeathCumulative', 'Per100kTestsCumulative', 'Date', 'DateTime']]

Midwest=['MO', 'KS', 'IA', 'NE', 'MN', 'IL', 'IN', 'OH', 'MI', 'WS']
midwest_df=df[df['state'].isin(Midwest)]
midwest_df.tail()
df.to_csv('AllStates.csv')
midwest_df.to_csv('Midwest.csv')

# Using seaborn for statistical displays

In [7]:
df=pd.read_csv('Results.csv')
df=df.filter(['File', 'Primary', 'Secondary', 'Spots1', 'Spots3', 'Area', 'Density1', 'Density3'], axis=1)
df=df.rename(columns={'Primary':'Condition', 'Secondary':'Time', 'Spots1':'TUNEL', 'Spots3':'H3p', 'Density1':'TUNEL_Density',
             'Density3':'H3p_Density'})
df['Time']=df['Time'].str.split('d').str[0]
df.set_index('File').to_csv('WormDataset.csv')
df.to_csv('WormDataset.csv')